# Animation Framework
*Arthur Ryman, lasted updated 2025-08-22*


## Introduction

These notes describe an idea for organizing
animation code into modules that each have a limited scope.

The purpose of an animation is to draw a sequence of pictures.
A picture can have many parts. For example, a cube has six faces,
each of which is a Polygon. 

A Manim Scene can contain many Mobject's.
A Mobject can contain many submobjects.
Without loss of generality, we can assume that a picture is a Mobject.

In [1]:
from manim import Mobject

type Picture = Mobject

/Users/arthurryman/Documents/repositories/GitHub/agryman/instant-insanity/venv/lib/python3.13/site-packages/manim_voiceover/__init__.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


A picture represents a single, static, frame
of a movie. Let Movie denote a list of pictures.

In [2]:
type Movie = list[Picture]

The state of the Picture will depend on some collection of variables.
We refer to this set of variables as a configuration.
We refer to the set of all configurations as the configuration space.

For example, a Circle depends on a centre point, a radius, and a
collection of settings such as fill_colour and stroke_colour.

Let C denote some configuration space type. 
C varies depending on the animation we are drawing. 

Think of C as a type variable. 
For the concrete example of a Circle we might define the parameter space CircleConfig
as follows:

In [3]:
from dataclasses import dataclass
from manim.typing import Point3D
from manim import ManimColor

@dataclass
class CircleConfig:
    radius: float
    centre: Point3D
    stroke_colour: ManimColor
    fill_colour: ManimColor

The whole point of defining a configuration space is that we can use it to create a Picture.
Therefore, we need to define a draw function that generates a Picture from a parameter.

For example, draw a Picture that contains one Circle.

In [4]:
from manim import Circle

def draw_circle_params(config: CircleConfig) -> Picture:
    circle: Circle = Circle(radius=config.radius,
                            fill_color=config.fill_colour,
                            stroke_color=config.stroke_colour)
    circle.move_to(config.centre)
    return circle

In general, a drawing function creates a Picture from some configuration space C.

In [5]:
from typing import Callable

type DrawPicture[C] = Callable[[C], Picture]

The DrawPicture type definition can be used by type checkers such as mypy.

So far we have the following ingredients:
* an arbitrary configuration space C, e.g. CircleParams
* drawing function f, of type DrawPicture[C], e.g. draw_p_circle

## Animations

Next, consider animations.
An animation is a movie in which successive pictures vary slightly so that when they are played back
we get the illusion of continuous motion.

Since we can draw any point in the configuration space, we can generate a movie from a continuous
path in the configuration space.
All movies have a start and a stop.
A path in the configuration space is simply a function that maps some finite interval of real numbers to
the configuration space.
Without loss of generality, we can assume the real numbers $\alpha$ lie in the unit interval $I = [0, 1]$
where $0$ corresponds to the start and $1$ to the stop.

Let Alpha be a type alias for float which we restrict to the unit interval.

In [6]:
type Alpha = float
type CircleConfigPath = Callable[[Alpha], CircleConfig]

For example, here is a path that uniformly varies the radius from 1.0 to 2.0
of a red circle, with black boundary, centered at the origin.

In [7]:
from manim import ORIGIN, RED, BLACK

def circle_radius_1_2(alpha: float) -> CircleParams:
    radius: float = 1.0 + alpha
    config: CircleConfig = CircleConfig(radius=radius,
                                        centre=ORIGIN,
                                        stroke_colour=BLACK,
                                        fill_colour=RED)
    return config

NameError: name 'CircleParams' is not defined


Of course, pictures will be drawn for only a finite number of values of $\alpha$.
The actual values of $\alpha$ generated depends on the run time and frame rate.
That level of detail is controlled by a renderer.
We can ignore those details.

A generic configuration path is parameterized by a configuration space C.

In [ ]:
type ConfigPath[C] = Callable[[Alpha], C]

A movie is therefore defined by the combination of a configuration path and a drawing function.
Here we assume a uniform sampling of Alpha consisting of n equally-spaced values.

In [ ]:
import numpy as np
from numpy.typing import NDArray

def make_movie[C](config_path: ConfigPath[C], draw_picture: DrawPicture[C], n: int) -> Movie:
    alphas: NDArray = np.linspace(0.0, 1.0, num=n, endpoint=False, dtype=float)
    movie: Movie = [draw_picture(config_path(alpha)) for alpha in alphas]
    return movie

Of course, make_movie() is just conceptual. It's what the Manim renderer does. When Manim renders a scene it runs
several animations, one after the other, looping over a time parameter and updating the Picture incrementally.

The point here is that an animation is conceptually composed of the following three main components:
* a configuration space C
* a drawing function f that computes a Picture given a configuration c in C
* a path function p that maps a real parameter alpha to the configuration space

These breakdown of the animation into these three parts will let us define classes that have well-defined
responsibilities.

The drawing function is very tightly associated with the configuration space.
However, there are many possible paths in configuration space.
For example, consider rigid motions in euclidean space.
Here the configuration space is the set of rotations and translations acting on euclidean space.
These parameters can be applied to any collection of objects embedded in euclidean space.

## Transformations

We frequently encounter the situation in which the configuration space decomposes into a pair consisting of an initial geometry g of Type G and a transformation t of
type T. The transformation acts on the set of geometries:

$$
t: G \rightarrow G
$$

In this case:
$$
C = T \times G
$$

We can evaluate the transform and a configuration to get another configuration.

$$
(t, g) \mapsto t(g)
$$

In [ ]:
type TransformationConfig[T,G] = tuple[T,G]

We further require that T contains an identity transformation e:

$$
\forall g: G \bullet e(g) = g
$$

Suppose we are given a path $\tau: [0,1] \rightarrow T$ in transformation space such that $\tau(0) = e$.
Suppose also that we are given a starting configuration $c_0 \in C$.
Then this defines a path $\gamma$ in C by:

$$
\gamma(\alpha) = (\tau(\alpha), \tau(\alpha)(c_0))
$$

with $\gamma(0) = (e, c_0)$

Furthermore, the drawing function typically only depends on the $G$ component of $C$.

## Classes

Above we used simple type definitions for the configuration space, the drawing functions, and the configuration paths.
However, these type will in general have a lot of internal structure so they should be implemented as classes.
Here are my proposed names for three abstract base classes.
* ConfigSpace: $C$
* DrawFunction: $C \rightarrow \mbox{Picture}$
* ConfigPath: $I \rightarrow C$